# IMDB 리뷰 감성 분류 : Word2Vec 활용 로지스틱 회귀 모델
- 데이터 전처리 : imdb_preprocessing.ipynb

## 라이브러리 임포트

In [1]:
import logging
import numpy as np
import pandas as pd
from gensim.models import word2vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DATA_PATH = '/content/drive/MyDrive/nlpdata/imdb/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

In [4]:
train_data = pd.read_csv(DATA_PATH + TRAIN_CLEAN_DATA)

In [5]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

## word2vev을 사용하기 위해 입력값을 단어로 구분된 리스트로 만듬 

In [6]:
sentences = []
for review in reviews:
    sentences.append(review.split())

## word2vec 학습 및 모델 저장
- gensim 라이브러리가 설치 안 되어 있으면 설치
- `!pip install gensim`

In [ ]:
num_features = 300      # 단어 임베딩 벡터 차원수
min_word_count = 40     # 적은 빈도의 수의 단어는 학습하지 않음
num_workers = 4         # 모델 학습 프로세스 개수
context = 10            # 컨텍스트 윈도 크기
downsampling = 1e-3     # 다운 샘플링 비율(보통 0.001)

model = word2vec.Word2Vec(sentences, workers=num_workers,
                          size=num_features, min_count=min_word_count,
                          window = context, sample=downsampling)

model_name = "300features_40minwords_10context"
model.save(DATA_PATH +model_name)

### 분류 모델 학습을 위해 입력값을 같은 형태로 만듬
각 리뷰에 있는 전체 단어의 평균값을 계산하는 함수
- words : 단어의 모음인 하나의 리뷰 데이터
- model : 학습된 word2vec 모델
- num_features : 임베딩 벡터 차원수
index2word_set 셋으로 문장의 단어가 모델 단어사전에 속하는지 확인

1. model.wv.index2word로 set 객체 생성
2. 반복문으로 임베딩된 벡터가 있는 단어 벡터의 합을 구함
3. 단어의 전체 개수로 나누어 평균 벡터의 값 계산

In [8]:
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

### 전체 리뷰에 대해 각 리뷰의 평균 벡터를 구하는 함수

In [9]:
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [ ]:
test_data_vecs = get_dataset(sentences, model, num_features)

## 학습과 검증 데이터셋 분리

In [11]:
X = test_data_vecs
y = np.array(sentiments)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 로지스틱 회귀 모델 선언 및 학습

In [ ]:
lr = LogisticRegression(class_weight='balanced')
lr.fit(X_train, y_train)
print(f"Accuracy: {lr.score(X_test, y_test):.2f}") 

In [13]:
TEST_CLEAN_DATA = 'test_clean.csv'
test_data = pd.read_csv(DATA_PATH + TEST_CLEAN_DATA)
test_review = list(test_data['review'])

In [14]:
test_sentences = list()
for review in test_review:
    test_sentences.append(review.split())

In [ ]:
test_data_vecs = get_dataset(test_sentences, model, num_features)

In [16]:
ids = list(test_data['id'])
test_predicted = lr.predict(test_data_vecs)

answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_PATH + 'answer_lr_w2v.csv', index=False, quoting=3)

## kgggle에 결과 제출 및 스코어 확인
- https://www.kaggle.com/c/word2vec-nlp-tutorial